In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns',None)#displaying long list of columns
pd.set_option('display.max_rows', None)#displaying long list of rows
pd.set_option('display.width', 1000)#width of window
import statsmodels.api as sm


file = r'/gdrive/My Drive/python3/Python 3 B regression/bankdf1.csv'
Data = pd.read_csv(file)  #creates a dataframe

print(Data.shape)


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
(515, 18)


In [ ]:
Data.drop(Data.columns[0], axis=1, inplace=True)


In [ ]:
list(Data.columns)

['CustomerID',
 'LoanPayoffPeriodInMonths',
 'LoanReason',
 'RequestedAmount',
 'InterestRate',
 'Co-Applicant',
 'YearsAtCurrentEmployer',
 'YearsInCurrentResidence',
 'Age',
 'RentOrOwnHome',
 'TypeOfCurrentEmployment',
 'NumberOfDependantsIncludingSelf',
 'CheckingAccountBalance',
 'DebtsPaid',
 'SavingsAccountBalance',
 'CurrentOpenLoanApplications',
 'WasTheLoanApproved']

In [ ]:
Data

,CustomerID,LoanPayoffPeriodInMonths,LoanReason,RequestedAmount,InterestRate,Co-Applicant,YearsAtCurrentEmployer,YearsInCurrentResidence,Age,RentOrOwnHome,TypeOfCurrentEmployment,NumberOfDependantsIncludingSelf,CheckingAccountBalance,DebtsPaid,SavingsAccountBalance,CurrentOpenLoanApplications,WasTheLoanApproved
0,741,36,goods,9566,2,none,4,2,31,owned,skilled,1,none,delayed,some,2,1
1,439,12,goods,2279,4,none,4,4,37,free,skilled,1,none,paid,none,1,1
2,501,30,auto,3857,4,none,4,4,40,owned,self,1,debt,paid,some,1,1
3,147,12,auto,9004,none,Unknown,4,2,23,owned,skilled,1,debt,paid,none,1,0
4,749,21,auto,2923,1,none,4,1,28,owned,self,1,high,paid,medium,1,1
5,468,9,goods,9184,none,Unknown,4,1,30,owned,skilled,1,some,paid,some,1,0
6,456,15,repairs,1512,3,none,4,3,61,owned,skilled,1,some,delayed,v.high,2,0
7,487,30,goods,4583,2,guarant,4,2,32,owned,skilled,1,debt,paid,some,2,1
8,504,30,auto,4811,2,none,7,4,24,rent,unskill,1,none,paid,none,1,1
9,688,18,goods,4333,none,co-app,0,4,22,rent,skilled,1,none,paid,some,1,0


In [ ]:
Data.dtypes
cat = ['LoanReason','InterestRate','Co-Applicant','YearsAtCurrentEmployer','RentOrOwnHome','TypeOfCurrentEmployment','CheckingAccountBalance','DebtsPaid','SavingsAccountBalance']


In [ ]:
y=Data['WasTheLoanApproved']
X=Data.drop(['WasTheLoanApproved'], axis=1)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
cat_columns = ['LoanReason','Co-Applicant','RentOrOwnHome','TypeOfCurrentEmployment','CheckingAccountBalance','DebtsPaid','SavingsAccountBalance']

df_processed = X_train[['CustomerID','LoanPayoffPeriodInMonths','RequestedAmount','Age','CurrentOpenLoanApplications', 'NumberOfDependantsIncludingSelf','InterestRate','YearsAtCurrentEmployer','YearsInCurrentResidence']]


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# For each categorical column
# We fit a label encoder, transform our column and 
# add it to our new dataframe
label_encoders = {}
for col in cat_columns:
    print("Encoding {}".format(col))
    new_le = LabelEncoder()
    df_processed[col] = new_le.fit_transform(X_train[col])
    label_encoders[col] = new_le

Encoding LoanReason
Encoding Co-Applicant
Encoding RentOrOwnHome
Encoding TypeOfCurrentEmployment
Encoding CheckingAccountBalance
Encoding DebtsPaid
Encoding SavingsAccountBalance


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
cat_columns_idx = [df_processed.columns.get_loc(col) 
                   for col in cat_columns]

In [ ]:
ohe = OneHotEncoder(categorical_features=cat_columns_idx, 
                    sparse=False, handle_unknown="ignore")
df_processed_np = ohe.fit_transform(df_processed)

TypeError: ignored

In [ ]:
Data['RequestedAmount']=1*Data['RequestedAmount'] 

Data.RequestedAmount = pd.to_numeric(Data.RequestedAmount, errors="coerce")

In [ ]:
Data.RequestedAmount.astype(int)


ValueError: ignored

In [ ]:
Data.dtypes

CustomerID                           int64
LoanPayoffPeriodInMonths             int64
LoanReason                          object
RequestedAmount                    float64
InterestRate                       float64
Co-Applicant                        object
YearsAtCurrentEmployer              object
YearsInCurrentResidence              int64
Age                                  int64
RentOrOwnHome                       object
TypeOfCurrentEmployment             object
NumberOfDependantsIncludingSelf      int64
CheckingAccountBalance              object
DebtsPaid                           object
SavingsAccountBalance               object
CurrentOpenLoanApplications          int64
WasTheLoanApproved                   int64
dtype: object

In [ ]:
Data['InterestRate'] = Data['InterestRate'].replace(['none'],'0')
Data['YearsAtCurrentEmployer'] = Data['YearsAtCurrentEmployer'].replace(['10+'],'10')

In [ ]:
Data.dtypes

NameError: ignored

In [ ]:
Data.InterestRate = pd.to_numeric(Data.InterestRate, errors="coerce")
Data.InterestRate.astype(int)

,CustomerID,LoanPayoffPeriodInMonths,LoanReason,RequestedAmount,InterestRate,Co-Applicant,YearsAtCurrentEmployer,YearsInCurrentResidence,Age,RentOrOwnHome,TypeOfCurrentEmployment,NumberOfDependantsIncludingSelf,CheckingAccountBalance,DebtsPaid,SavingsAccountBalance,CurrentOpenLoanApplications,WasTheLoanApproved
0,741,36,goods,9566.0,2,none,4,2,31,owned,skilled,1,none,delayed,some,2,1
1,439,12,goods,2279.0,4,none,4,4,37,free,skilled,1,none,paid,none,1,1
2,501,30,auto,3857.0,4,none,4,4,40,owned,self,1,debt,paid,some,1,1
3,147,12,auto,9004.0,0,Unknown,4,2,23,owned,skilled,1,debt,paid,none,1,0
4,749,21,auto,2923.0,1,none,4,1,28,owned,self,1,high,paid,medium,1,1
5,468,9,goods,9184.0,0,Unknown,4,1,30,owned,skilled,1,some,paid,some,1,0
6,456,15,repairs,1512.0,3,none,4,3,61,owned,skilled,1,some,delayed,v.high,2,0
7,487,30,goods,4583.0,2,guarant,4,2,32,owned,skilled,1,debt,paid,some,2,1
8,504,30,auto,4811.0,2,none,7,4,24,rent,unskill,1,none,paid,none,1,1
9,688,18,goods,4333.0,0,co-app,0,4,22,rent,skilled,1,none,paid,some,1,0


In [ ]:
Data.YearsAtCurrentEmployer.astype(int)

0       4
1       4
2       4
3       4
4       4
5       4
6       4
7       4
8       7
9       0
10      4
11     10
12     10
13      7
14     10
15      1
16     10
17     10
18     10
19      0
20      4
21      7
22     10
23     10
24      4
25      1
26      7
27     10
28     10
29     10
30      7
31     10
32      1
33     10
34      4
35      4
36      0
37      7
38      4
39      4
40      4
41      7
42     10
43     10
44      4
45      4
46      7
47      4
48      7
49      4
50      7
51      1
52      4
53      4
54      7
55      4
56      4
57     10
58      4
59      1
60      1
61      4
62      7
63      4
64      4
65     10
66     10
67     10
68      1
69     10
70     10
71      7
72      7
73     10
74      4
75      1
76      0
77      7
78      1
79     10
80      7
81      1
82      1
83      4
84      7
85      4
86     10
87      7
88      4
89      4
90      4
91      0
92      4
93     10
94     10
95     10
96      1
97      4
98      4
99      1


In [ ]:
Data.YearsAtCurrentEmployer = pd.to_numeric(Data.YearsAtCurrentEmployer, errors="coerce")
Data.YearsAtCurrentEmployer.astype(int)

0       4
1       4
2       4
3       4
4       4
5       4
6       4
7       4
8       7
9       0
10      4
11     10
12     10
13      7
14     10
15      1
16     10
17     10
18     10
19      0
20      4
21      7
22     10
23     10
24      4
25      1
26      7
27     10
28     10
29     10
30      7
31     10
32      1
33     10
34      4
35      4
36      0
37      7
38      4
39      4
40      4
41      7
42     10
43     10
44      4
45      4
46      7
47      4
48      7
49      4
50      7
51      1
52      4
53      4
54      7
55      4
56      4
57     10
58      4
59      1
60      1
61      4
62      7
63      4
64      4
65     10
66     10
67     10
68      1
69     10
70     10
71      7
72      7
73     10
74      4
75      1
76      0
77      7
78      1
79     10
80      7
81      1
82      1
83      4
84      7
85      4
86     10
87      7
88      4
89      4
90      4
91      0
92      4
93     10
94     10
95     10
96      1
97      4
98      4
99      1


In [ ]:
Data.dtypes

CustomerID                           int64
LoanPayoffPeriodInMonths             int64
LoanReason                          object
RequestedAmount                    float64
InterestRate                       float64
Co-Applicant                        object
YearsAtCurrentEmployer               int64
YearsInCurrentResidence              int64
Age                                  int64
RentOrOwnHome                       object
TypeOfCurrentEmployment             object
NumberOfDependantsIncludingSelf      int64
CheckingAccountBalance              object
DebtsPaid                           object
SavingsAccountBalance               object
CurrentOpenLoanApplications          int64
WasTheLoanApproved                   int64
dtype: object

In [ ]:
Data

,CustomerID,LoanPayoffPeriodInMonths,LoanReason,RequestedAmount,InterestRate,Co-Applicant,YearsAtCurrentEmployer,YearsInCurrentResidence,Age,RentOrOwnHome,TypeOfCurrentEmployment,NumberOfDependantsIncludingSelf,CheckingAccountBalance,DebtsPaid,SavingsAccountBalance,CurrentOpenLoanApplications,WasTheLoanApproved
0,741,36,goods,9566.0,2.0,none,4,2,31,owned,skilled,1,none,delayed,some,2,1
1,439,12,goods,2279.0,4.0,none,4,4,37,free,skilled,1,none,paid,none,1,1
2,501,30,auto,3857.0,4.0,none,4,4,40,owned,self,1,debt,paid,some,1,1
3,147,12,auto,9004.0,0.0,Unknown,4,2,23,owned,skilled,1,debt,paid,none,1,0
4,749,21,auto,2923.0,1.0,none,4,1,28,owned,self,1,high,paid,medium,1,1
5,468,9,goods,9184.0,0.0,Unknown,4,1,30,owned,skilled,1,some,paid,some,1,0
6,456,15,repairs,1512.0,3.0,none,4,3,61,owned,skilled,1,some,delayed,v.high,2,0
7,487,30,goods,4583.0,2.0,guarant,4,2,32,owned,skilled,1,debt,paid,some,2,1
8,504,30,auto,4811.0,2.0,none,7,4,24,rent,unskill,1,none,paid,none,1,1
9,688,18,goods,4333.0,0.0,co-app,0,4,22,rent,skilled,1,none,paid,some,1,0


In [ ]:
export_csv = Data.to_csv(r'/gdrive/My Drive/python3/Python 3 B regression/inner join.csv')